### Pyspark initialization


In [1]:
#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/18 21:52:59 WARN Utils: Your hostname, Rudyletsgo resolves to a loopback address: 127.0.1.1; using 172.26.136.198 instead (on interface eth0)
22/09/18 21:52:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/18 21:53:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import countDistinct

In [3]:
raw_data_path = "../data/tables/"

In [4]:
# need to install xlrd run pip install xlrd
poa_2011_2016 = pd.read_excel(raw_data_path+"CG_POA_2011_POA_2016.xls",sheet_name = "Table 3", header = 5)
sa2_2011_2016 = pd.read_excel(raw_data_path+"CG_SA2_2011_SA2_2016.xls",sheet_name = "Table 3", header = 5)
poa_sa2_2011 = pd.read_excel(raw_data_path+"1270055006_CG_POSTCODE_2011_SA2_2011.xls",sheet_name = "Table 3", header = 5)
poa_sa2_2016 = pd.read_excel(raw_data_path+"CG_POA_2016_SA2_2016.xls",sheet_name = "Table 3", header = 5)

Drop the null column caused by type transform

In [5]:
# change to pyspark dataframe
poa_2011_2016.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
sa2_2011_2016.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
poa_sa2_2011.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
poa_sa2_2016.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [6]:
poa_sa2_2011['SA2_MAINCODE_2011'] = poa_sa2_2011['SA2_MAINCODE_2011'].apply(np.int64)
poa_sa2_2016['SA2_MAINCODE_2016'] = poa_sa2_2016['SA2_MAINCODE_2016'].apply(np.int64)
sa2_2011_2016['SA2_MAINCODE_2016'] = sa2_2011_2016['SA2_MAINCODE_2016'].apply(np.int64)

In [7]:
# Change the form to pyspark dataframe
poa_2011_2016 = spark.createDataFrame(poa_2011_2016) 
sa2_2011_2016 = spark.createDataFrame(sa2_2011_2016) 
poa_sa2_2011 = spark.createDataFrame(poa_sa2_2011) 
poa_sa2_2016 = spark.createDataFrame(poa_sa2_2016) 

Drop useless columns

In [8]:
poa_2016_2021 = spark.read.csv('../data/tables/poa_2016_2021.csv',header=True)
sa2_2016_2021 = spark.read.csv('../data/tables/sa2_2016_2021.csv',header=True)

In [9]:
common_cols = ['RATIO','PERCENTAGE']
poa_2011_2016 = poa_2011_2016.drop(*common_cols)
sa2_2011_2016 = sa2_2011_2016.drop(*common_cols)
poa_sa2_2011 = poa_sa2_2011.drop(*common_cols)
poa_sa2_2016 = poa_sa2_2016.drop(*common_cols)

In [10]:
common_cols_new = ['RATIO_FROM_TO', 'NDIV_TO_REGION_QLTY_INDICATOR', 'OVERALL_QUALITY_INDICATOR', 'BMOS_NULL_FLAG', 'INDIV_TO_REGION_QLTY_INDICATOR']
poa_2016_2021 = poa_2016_2021.drop(*common_cols_new)
sa2_2016_2021 = sa2_2016_2021.drop(*common_cols_new)


Check how many postcode left if using 2011 correspondence

In [11]:
table_2011 = poa_2011_2016.join(poa_sa2_2011,poa_2011_2016['POA_CODE_2011'] == poa_sa2_2011['POSTCODE'], how = 'inner')

In [12]:
table_2011 = table_2011.join(sa2_2011_2016, table_2011['SA2_MAINCODE_2011'] == sa2_2011_2016['SA2_MAINCODE_2011'], how = 'inner')

In [13]:
poa_2016_2021

POA_CODE_2016,POA_NAME_2016,POA_CODE_2021,POA_NAME_2021
0800,0800,0800,0800
0810,0810,0810,0810
0812,0812,0812,0812
0815,0815,0810,0810
0820,0820,0820,0820
0822,0822,0837,0837
0822,0822,0822,0822
0828,0828,0828,0828
0829,0829,0829,0829
0830,0830,0830,0830


In [14]:
table_2011 = table_2011.join(sa2_2016_2021, table_2011['SA2_MAINCODE_2016'] == sa2_2016_2021['SA2_MAINCODE_2016'], how = 'inner')
table_2011 = table_2011.join(poa_2016_2021, table_2011['POA_CODE_2016'] == poa_2016_2021['POA_CODE_2016'], how = 'inner')

In [16]:
table_2011_new = table_2011['POA_CODE_2021','SA2_CODE_2021','SA2_NAME_2021']

In [17]:
distinct_df_2011 = table_2011_new.distinct()

In [18]:
distinct_df_2011.select(countDistinct("POA_CODE_2021"))

count(DISTINCT POA_CODE_2021)
2638


Check how many postcode left if using 2016 correspondence

In [23]:
table_2016 = poa_2016_2021.join(poa_sa2_2016, poa_2016_2021['POA_CODE_2016'] == poa_sa2_2016['POA_CODE_2016'], how = 'inner')
table_2016 = table_2016.join(sa2_2016_2021, table_2016['SA2_MAINCODE_2016'] == sa2_2016_2021['SA2_MAINCODE_2016'], how = 'inner')

In [26]:
table_2016_new = table_2016['POA_CODE_2021','SA2_CODE_2021','SA2_NAME_2021']

In [29]:
distinct_df_2016 = table_2016_new.distinct()
distinct_df_2016.select(countDistinct("POA_CODE_2021"))

count(DISTINCT POA_CODE_2021)
2641


Therefore use the postcode and sa2 linkage based on 2016 correspondence

In [30]:
distinct_df_2016.write.mode('overwrite').option("header",True).csv('../data/curated/postcode_SA2.csv')